In [1]:
# Using this because setup.py is not working
import sys; sys.path.append("../../")

import sys; sys.path.append("../data")

import measurements_data
import patient_data
import antibiotics_data
import O2_FEV1_df

from partition import *
from desaturation_fev1 import *
from normalise import *
from exacerbation_periods import *
from smooth import *
from plot_helpers import *
from biology import *
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go


#Settings
plotsdir="../../../../PlotsSmartcare/O2_FEV1/"

In [2]:
# Process O2-FEV1 data
O2_FEV1 = O2_FEV1_df.create()

# Compute FEV1 % Predicted
O2_FEV1['FEV1 % Predicted'] = O2_FEV1.apply(lambda x: x["FEV1"] / x["Predicted FEV1"] * 100, axis=1)

# Partition FEV1 % Predicted in 3 groups
O2_FEV1["FEV1 % Predicted Group"] = partition_in_n_equal_groups(O2_FEV1['FEV1 % Predicted'],3)

# What are we analysing?
O2_FEV1.head()


** Loading measurements data **


/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/O2-FEV1 analysis/../data/measurements_data.py:8: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(datadir + "mydata.csv")



* Merging measurements data with ID map to retrieve SmartCare ID *
List User IDs that have no SmartCare ID ['o0egatM37za2ZZ2jfs3AJ5' '-Q0Wf614z94DSTy6nXjyw7'
 '31KnMK5Z4A6OKErB1C5-i6' '0HeWh64M_zc5U5l2xqzAs4'
 'GyK_hmIGxVaT7chwlGDio7' '1au5biSTt0bNWgfI0WItr5'
 '-TKpptiCA5cASNKU0VSmx4' '-Cujq-NEcld_Keu_W1-Nw5'
 'ci0iszS3ov6G2U-dKDiAE4' '9YevbyBS8VdwRL3RRUGY86'
 'MB0_-sqQBxhrKyfoOxUaF4' '9MKCpOe_FO_LcU9C0oCPJ4']
6694 rows with User ID from measurements data not present in Patient_ID from ID map

Generating fake SmartCare IDs for those User IDs, SmartCare IDs are within 23-241 range, we'll generate fake IDs starting 300.
All IDs sorted: [ 23  24  27  28  29  30  31  32  33  34  35  36  37  39  40  41  42  43
  44  45  46  47  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  78  79  80  81  82  88  92
  93  99 100 102 107 113 114 115 121 122 123 124 126 127 128 129 130 131
 132 133 134 135 136 137 138 139 140 141 142 143 144 151 152 153 

/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/O2-FEV1 analysis/../data/measurements_data.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_map = id_map.append(


O2 Saturation sanity check
IDs with O2 Saturation outside 70-100 % range: 
        ID  O2 Saturation
52484  128          127.0
53781  129          127.0
55379  129          127.0
46706   23          127.0
46119   24          127.0
50403   28          127.0
47510   32          127.0
49622   46          127.0
49087   55          127.0
53923   57           48.0
46782   82          104.0
47102   82          103.0
Removing 12 rows with O2 Saturation outside 70-100 % range

* Looking for duplicates *
Found 4649 duplicates, saving them in DataFiles/SmartCare/duplicates.xlsx
Removing 4649 duplicated entries

Loaded measurements data with 125180 entries (initially 129841, removed 4661)

** Loading patient data **

* Dropping unnecessary columns from patient data *
Columns filtered: ['ID', 'Study Date', 'DOB', 'Age', 'Sex', 'Height', 'Weight', 'Predicted FEV1', 'FEV1 Set As']
Columns dropped: {'Inconvenience Payment', 'Freezer Required', 'Comments', 'CFQR Quest Comp', 'Date Last PE Stop', 'Date 

/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


ID 60: Corrected height 60 from 1.63 to 163.0
ID 66: Corrected height for ID 66 from 1.62 to 162.0
Replace Age by calculate age
Replace Predicted FEV1 by the calculated version
Drop FEV1 Set As

* Applying data sanity checks *
Loaded patient data with 147 entries (147 initially)

** Loading antibiotics data **

* Dropping unnecessary columns from antibiotics data *
Columns filetered: ['ID', 'Antibiotic Name', 'Route', "Home IV's'", 'Start Date', 'Stop Date']
Columns dropped: {'Study Number', 'Hospital', 'Antibiotic ID'}

** Creating DataFrame for O2 FEV1 analysis **
O2 Saturation has 14026 measurements
FEV1 has 9949 measurements
Merged O2 and FEV1 into 9537 entries (initially 14438, removed 4901 NaN)
Asserting that there's only one measurement per ID per day

Created df_O2_FEV1 with 9537 entries (initially 14438, removed 4901)


,ID,Date recorded,O2 Saturation,FEV1,Study Date,DOB,Age,Sex,Height,Weight,Predicted FEV1,FEV1 % Predicted,FEV1 % Predicted Group
0,23,2015-08-12,95.0,1.49,2015-08-03,1960-07-05,55.0,Female,157.0,63.4,2.2265,66.921177,>=61.2
1,138,2015-08-14,94.0,1.30,2015-08-06,1985-10-31,30.0,Male,172.0,66.6,4.0360,32.210109,<41.8
2,23,2015-08-14,95.0,1.29,2015-08-03,1960-07-05,55.0,Female,157.0,63.4,2.2265,57.938468,[41.8;61.2[
3,23,2015-08-21,94.0,1.35,2015-08-03,1960-07-05,55.0,Female,157.0,63.4,2.2265,60.633281,[41.8;61.2[
4,137,2015-08-26,95.0,1.28,2015-08-25,1976-05-24,39.0,Female,172.0,61.0,3.2190,39.763902,<41.8


# O2-FEV1 analysis
## Definitions
- O2 Saturation
- FEV 1
- Predicted FEV 1 in %: measure for the amount of airway obstruction either due to sputum load (non-permanent obstruction) or scars in the lungs (permanent obstruction).

## Literature
[The association between forced expiratory volume in one second (FEV1) and pulse oximetric measurements of arterial oxygen saturation (SpO2) in the patients with COPD: A preliminary study](https://pubmed.ncbi.nlm.nih.gov/24949035/).
- Context: 31 patients with COPD
- Key results: There was not statistically significant correlation between FEV1 % predicted and SpO2 values (P < 0.05), but a great correlation existed between FEV1/FVC % predicted and SpO2 values (r = 0.556, P < 0.001).

 [ANALYSIS OF CORRELATION BETWEEN FEV1/FEV6 AND OXYGEN SATURATION DURING SIX-MINUTE WALK TEST (6MWT) IN COPD PATIENTS](https://www.researchgate.net/publication/351322676_ANALYSIS_OF_CORRELATION_BETWEEN_FEV1FEV6_AND_OXYGEN_SATURATION_DURING_SIX-MINUTE_WALK_TEST_6MWT_IN_COPD_PATIENTS)
 - Need to download

Note: nobody segments the input by groups. Probably because they don't have enough data to do this.

## Raw scatter plots

In [4]:
# Raw plot without trendline
fig = px.scatter(O2_FEV1, y="O2 Saturation", x="FEV1")
# fig = px.scatter(O2_FEV1, y="O2 Saturation", x="FEV1", trendline="ols")
fig.update_layout(autosize=False, width=500, height=500)
fig.show()
fig.write_image(plotsdir + "FEV1-O2 raw.pdf")

fig = px.scatter(O2_FEV1, y="O2 Saturation", x="FEV1 % Predicted")
fig.update_layout(autosize=False, width=500, height=500)
fig.show()
fig.write_image(plotsdir + "FEV1 % Predicted-O2 raw.pdf")

O2_FEV1.shape

(9537, 13)

In [7]:
var = 'FEV1'; xaxis_range = [0, 5.1]
var = 'FEV1 % Predicted'; xaxis_range = [0, 155]

# Plot with height
O2_FEV1['Height Group'], height_labels=partition_in_n_equal_groups(O2_FEV1['Height'], 3, True)

fig = px.scatter(O2_FEV1, y="O2 Saturation", x=var, color="Height Group", marginal_x="histogram", marginal_y="histogram")
# fig.update_layout(autosize=False, width=800, height=800)
fig.update_traces(marker=dict(size=5),
                  selector=dict(mode='markers'))
# fig.update_xaxes(range=[0.9*min(O2_FEV1["FEV1"]), 1.1*max(O2_FEV1["FEV1"])])
# fig.update_yaxes(range=[0.9*min(O2_FEV1["O2 Saturation"]), max(1.1*O2_FEV1["O2 Saturation"])])
# fig.show()

for height_group in height_labels:
    mask_height=O2_FEV1['Height Group'] == height_group

    fig = px.scatter(O2_FEV1[mask_height], y="O2 Saturation", x=var,
                     title='Height group {}'.format(height_group))
    fig.update_layout(autosize=False, width=500, height=500)
    fig.update_traces(marker=dict(size=5),
                      selector=dict(mode='markers'))
    fig.update_xaxes(range=xaxis_range)
    fig.update_yaxes(range=[74, 102])
    fig.show()
    filename= "Height group " +  height_group + " " + var + "-O2 raw"
    fig.write_image(plotsdir + filename + ".pdf")

In [10]:
predicted = False
x_var = 'Height'
lung_function_var = 'FEV1 % Predicted' if predicted else 'FEV1'

fig1, fig2, _ = desaturation_FEV1_for_variable(O2_FEV1, x_var, n_var_groups=6, predicted=predicted)
fig1.show()
fig1.write_image(plotsdir + "Desaturation {} for {} main plot.pdf".format(lung_function_var, x_var))

fig2.show()
fig2.write_image(plotsdir + "Desaturation {} for {} with lines.pdf".format(lung_function_var, x_var))

# Observations
# o2 10, var 15 contains a lot of noise and groups have very low number of datapoints. It gets too patient specific

## Per patient plots

In [ ]:
# Same patient variability: can we observe the same pattern for each patient?
O2_FEV1.head()

# Include time evolution into the graph
def to_float(dt_time):
    return (365*dt_time.year + 12*dt_time.month + dt_time.day)/12

mindate = to_float(min(O2_FEV1["Date recorded"]))
O2_FEV1["Months since study start"]=O2_FEV1["Date recorded"].apply(lambda x: to_float(x)-mindate)

In [ ]:
# Same patient variability: can we observe the same pattern for each patient?
IDs=O2_FEV1.ID.unique()
for ID in IDs:
    plot_o2_fev1_raw_for_id(O2_FEV1, ID)

## Summary plot (+ partitioning)

In [3]:
# Add exacerbated labels
# Compute exacerbation labels
# O2_FEV1_processed = compute_ex_labels_from_heuristics(antibioticsdata, patientsdata, O2_FEV1)

# Get exacerbation labels from the predictive classifier
O2_FEV1_processed = merge_pred_ex_labels_to(O2_FEV1, get_pred_ex_labels())

# Smooth
O2_FEV1_processed = smooth(O2_FEV1_processed, ['FEV1', 'FEV1 % Predicted'], mode="max")
O2_FEV1_processed = smooth(O2_FEV1_processed, ['O2 Saturation'], mode="mean")
print("Removed {} nan entries after max smoothing, kept {}".format(O2_FEV1_processed.isna().sum().max(), O2_FEV1_processed.shape[0]))
O2_FEV1_processed.dropna(inplace=True)

# Normalise
O2_FEV1_processed = norm_by_stable_baseline(O2_FEV1_processed,
                                            ['O2 Saturation', 'O2 Saturation smoothed', 'FEV1', 'FEV1 % Predicted', 'FEV1 smoothed', 'FEV1 % Predicted smoothed'])

# Settings
prefix, ex_column, x, y = set_x_y_vars(
      with_predicted_labels=True,
      with_predicted_fev1=False,
      is_smoothed_fev=False,
      is_smoothed_o2=False,
      is_normalised=False,
  )

is_partitioned= False
if not is_partitioned:
  # Plot O2-FEV with respective displots
  fig=plot_o2_fev_with_displots(O2_FEV1_processed, x, y, ex_column)
  fig.show()
  fig.write_image("{}/Ex labels {} {}-{}.pdf".format(plotsdir, prefix, x, y), width=1400, height=600)

  # Plot subsampled O2-FEV scatter
  fig = plot_subsampled_scatter(x, y, O2_FEV1_processed, random_state=7)
  fig.show()
  
else:
  # Use partitioned data. Note that this requires smoothed True and Predicted FEV1 False

  predicted_fev1_thresholds = [2.4, 3.2, 4]

  # Print number of uniaue ID
  print("There are {} different individuals".format(len(O2_FEV1_processed.ID.unique())))

  # Filter by unique ID
  O2_FEV1_processed = O2_FEV1_processed.groupby('ID').filter(lambda l: len(l) > 1)
  # Check if individual's Predicted FEV1 position wrt to groups thresholds
  O2_FEV1_processed['Predicted FEV1 Group'] = O2_FEV1_processed['Predicted FEV1'].apply(lambda l: value_to_group(l, predicted_fev1_thresholds))

  # Create a for loop for each Group
  # parition_variable = 'FEV1 % Predicted smoothed'
  parition_variable = 'Predicted FEV1'

  for fev_group in O2_FEV1_processed[parition_variable + ' Group'].unique():
    mask = O2_FEV1_processed[parition_variable + ' Group'] == fev_group
    # Count patients
    n_patients=len(O2_FEV1_processed[mask].ID.unique())

    # Plot scatter plot using plotly with O2_FEV1_processed[mask], x, y, ex_column with color from get_ex_color() and get_stable_color()
    fig = px.scatter(O2_FEV1_processed[mask], x=x, y=y, color=ex_column, color_discrete_sequence=[get_stable_color(), get_ex_color()])
    # Set x axes range to min max of O2_FEV1_processed[x]
    fig.update_xaxes(range=[O2_FEV1_processed[x].min()*0.95, O2_FEV1_processed[x].max()*1.05])
    fig.update_layout(title="{} patients, {} {}".format(n_patients, parition_variable, fev_group))

    fig.show()
    fig.write_image("{}/Ex labels {} {}-{} group {}.pdf".format(plotsdir, prefix, x, y, fev_group), width=1400, height=600)

# Save O2_FEV1_processed to excel file in plotsdir
O2_FEV1_processed.to_excel(plotsdir + "O2_FEV1_processed.xlsx")

Exacerbated labels data from the predictive classifier has 14452 entries (1445 exacerbated, 13006 not exacerbated measurements, 1 NaN)
Merging exacerbated labels into O2_FEV1
Made inner merge of O2_FEV1 and exacerbated labels on 'ID' and 'Date recorded', leaving 7557 entries (initially 9537 in O2_FEV1, 14452 in pred_ex_labels
Removed 1 nan entries after max smoothing, kept 7557


## Test the sensitivity of random states on subsampled plots
We wonder if different random states will give largely different results, to an extend that it could challenge the model assumptions.
It turns out that, despite an effect on the x axis range, the subsampled plots are very similar.

In [ ]:
# Add subsampled plot
prefix, ex_column, x, y = set_x_y_vars(with_predicted_labels=True, with_predicted_fev1=True, is_smoothed=True, is_normalised=False)

for random_state in range(1,10):
  print(random_state)
  fig = plot_subsampled_scatter(x, y, O2_FEV1_processed, random_state=random_state)
  fig.show()

## Full sized displots

In [ ]:
# This part fo the code is unused now
# But the displots can be useful for the report, so I keep it here

if predicted:
    if normalised:
        xrange=[-1.8, 1.2]
    else:
        xrange=[0, 155]
    bin_size=4
else:
    if normalised:
        xrange=[-42,40]
    else:
        xrange=[0, 5.1]
    bin_size=0.14

# Raw plot with superposed exacerbation labels
fig = px.scatter(O2_FEV1_processed, y=y, x=x, color=ex_column)
# fig.update_layout(autosize=False, width=800, height=800)
fig.update_traces(marker=dict(size=5),
                  selector=dict(mode='markers'))
# fig.show()
# fig.write_image("{}/{}-{} by exacerbated labels superposed raw.pdf".format(plotsdir, x, y))

# Raw individual plot for each exacerbation label]
fig_scatter = px.scatter(O2_FEV1_processed.dropna(), y=y, x=x,
                 # color='Exacerbation Labels',
                 facet_col=ex_column)
fig_scatter.update_layout(autosize=False, width=1000, height=500)
fig_scatter.update_traces(marker=dict(size=5),
                  selector=dict(mode='markers'))
# fig_scatter.update_xaxes(range=xrange)
if not(normalised): fig_scatter.update_yaxes(range=[74, 102])
# fig_scatter.show()
# fig.write_image("{}/Ex labels {} {}-{} individual raw.pdf".format(plotsdir, prefix, x, y), width=1300, height=400)

def distplot(x, group_labels, bin_size):
    data=[]
    for exacerbation_label in group_labels:
        mask = O2_FEV1_processed[ex_column] == exacerbation_label
        data.append(O2_FEV1_processed[x][ mask ])
        print("{} measurements with {} = {}".format(sum(mask), ex_column, exacerbation_label))
    fig = ff.create_distplot(data, group_labels, bin_size=bin_size, show_rug=False)
    fig.update_xaxes(title_text=x)
    fig.update_layout(autosize=False, width=1000, height=400)
    return fig

# Create distplot for x = lung function
group_labels = [True, False] if with_predicted_labels else ["Exacerbation Period", "Stable Period"]
fig_distplot_fev1 = distplot(x, group_labels, bin_size)
fig_distplot_fev1.show()
# fig.write_image("{}/Ex labels {} {} distplot.pdf".format(plotsdir, prefix, x), width=1000, height=400)

# Create distplot for y = O2 saturation
group_labels = [True, False] if with_predicted_labels else ["Exacerbation Period", "Stable Period"]
fig_displot_o2 = distplot(y, group_labels, 1)
fig_displot_o2.show()
# fig_displot_o2.write_image("{}/Ex labels {} {} distplot.pdf".format(plotsdir, prefix, y), width=1000, height=400)

## Patient level plots to validate or validate outliers down

In [ ]:
# O2-FEV1: Outliers down can happen, however if you are persistently getting low reading it means that it's not an outlier
# Checking O2 sat stable period measurements outliers
exacerbation_label = 'Stable Period'

# Case 1:
IDs = ['180', '202']
o2_saturation_threshold = 87

# Case 2: smoothed
o2_saturation_threshold = 90

# Plot
ids = O2_FEV1_processed[ (O2_FEV1_processed['Exacerbation Labels'] == exacerbation_label) & (O2_FEV1_processed['O2 Saturation'] < o2_saturation_threshold) ].ID.unique()
print(ids)
for id in ids:
    plot_o2_fev1_raw_for_id(O2_FEV1_processed, id, save=False, show=True)

## Validate smooth max

In [ ]:
def plot_fev1_and_fev1_smoothed_for_id(O2_FEV1, id):
  O2_FEV1=O2_FEV1[ O2_FEV1['ID'] == id ]
  
  mode="max"
  O2_FEV1 = smooth(O2_FEV1, ['FEV1'], mode=mode)

  # Plot
  # Create a subplots that share the same x and y axes with plotly
  fig = make_subplots(rows=1, cols=1, shared_xaxes=True, shared_yaxes=True)
  # Add scatter of FEV1 with Date recorded with a name 'FEV1' with marker opacity
  fig.add_scatter(x=O2_FEV1['Date recorded'], y=O2_FEV1['FEV1'], mode='markers+lines', name='FEV1', opacity=0.5, row=1, col=1)
  # Add scatter of FEV1 smoothed with Date recorded with a name 'FEV1 smoothed'
  fig.add_scatter(x=O2_FEV1['Date recorded'], y=O2_FEV1['FEV1 smoothed'], mode='markers+lines', name='FEV1 smoothed', opacity=0.5, row=1, col=1)
  # Update figure size
  fig.update_layout(autosize=False, width=1000, height=300, title='FEV1 and FEV1 smoothed for ID {}'.format(id))
  fig.show()

# Plot plot_fev1_and_fev1_smoothed_for_id for the first 10 IDs of O2_FEV1
for id in O2_FEV1.ID.unique()[:10]:
  plot_fev1_and_fev1_smoothed_for_id(O2_FEV1, id)

## How to smooth O2?

In [ ]:
# Plots O2 saturation against Date for a given patient id
def plot_o2_saturation_for_id(df, id, plotsdir, save=False, show=False):
    mask = df['ID'] == id
    fig = px.scatter(df[mask], x='Date recorded', y='O2 Saturation')
    fig.update_layout(autosize=False, width=1000, height=400)
    fig.update_traces(marker=dict(size=5),
                      selector=dict(mode='markers'))
    if save:
        fig.write_image("{}/O2 for ID/O2 saturation for ID {}.pdf".format(plotsdir, id), width=1000, height=400)
    if show:
        fig.show()
        
# For each patient id, plot O2 saturation against Date recorded
for id in O2_FEV1.ID.unique():
    plot_o2_saturation_for_id(O2_FEV1, id, plotsdir, save=True, show=False)

## Why are there measurements with FEV1 % Pred smoothed > 70% and FEV1 smoothed < 1L?
image.png
The scatter plot of O2 vs FEV1 for the FEV1 % Predicted group between 70% and 90% shows several measurements with an FEV1 smoothed below 1L. This looks erroneous.
There are 98 measurements with FEV1 % Pred smoothed > 70% and FEV1 smoothed < 1L. 
- 92 measurements belong to individual with ID 216. There are no other measurements for this individual. Is there a problem for the data collected by this individual?

Action: add function `remove_all_measurements_with_ID_216()`

In [ ]:
114 * 0.01 * 3.95 - 44 * 0.025 - 2.6

In [ ]:
# Find rows where FEV1 % Predicted smoothed > 70% and FEV1 smoothed < 1 L
mask = (O2_FEV1_processed['FEV1 % Predicted smoothed'] > 70) & (O2_FEV1_processed['FEV1 smoothed'] < 1)
print("{} measurements with FEV1 % Predicted smoothed > 70% and FEV1 < 1 L".format(sum(mask)))
# Set pandas display max row option to 100
# pd.set_option('display.max_rows', 100)
# Apply the mask and filter the folowing columns ID, FEV1, FEV1 smoothed, Age, Sex, Height, Weight, FEV1 % Predicted, FEV1 % Predicted smoothed, Is Exacerbated
O2_FEV1_processed[mask][['ID', 'Age', 'Sex', 'Height', 'Weight', 'FEV1', 'FEV1 smoothed', 'Predicted FEV1', 'FEV1 Set As', 'FEV1 % Predicted', 'FEV1 % Predicted smoothed', 'Is Exacerbated', 'O2 Saturation']].sort_values(by=['ID', 'FEV1 % Predicted smoothed'])

In [ ]:
# Filter measurements done by ID 216
mask = O2_FEV1_processed['ID'] == '216'

# Add scatter of FEV1 smoothed against date recorded
fig = px.scatter(O2_FEV1_processed[mask], x='Date recorded', y='FEV1 smoothed')
# Add scatter of FEV1 with trace name FEV1
fig.add_scatter(x=O2_FEV1_processed[mask]['Date recorded'], y=O2_FEV1_processed[mask]['FEV1'], mode='markers', name='FEV1')
fig.show()
# Apply the mask and filter the folowing columns ID, FEV1, FEV1 smoothed, Age, Sex, Height, Weight, FEV1 % Predicted, FEV1 % Predicted smoothed, Is Exacerbated
O2_FEV1_processed[mask][['ID', 'Date recorded', 'FEV1', 'FEV1 smoothed', 'Age', 'Sex', 'Height', 'Weight', 'FEV1 % Predicted', 'FEV1 % Predicted smoothed', 'Is Exacerbated']].sort_values(by=['FEV1 smoothed'])

## Does the predictive classifier data contain smartcare IDs that are missing in the measurement data?

# Create Factor Functions
We want the characterise the factor function that links the Unblocked FEV1 with its parents: Healthy FEV1 and Lung Damage. Here's a model of the relations betweeen those three varaibles: the unblocked FEV1 (L) of an individual is the healthy FEV1 (L), the theoretical lung function based on height, DOB, gdner, ethnicity (TBC exactly which), pejorated by the % in lung damage

In [ ]:
# Factor function for unblocked FEV1 (L)
df_unblocked_factor = load_patient_data(datadir)

# For each patient id
# Create np.array to store the unblocked FEV1 (L)
rmax = []
for id in O2_FEV1.ID.unique():
  mask = O2_FEV1['ID'] == id
  # Find the unblocked FEV1 (L). We assume that, over the 6 months study period, the patient has done some measurements where he was not blocked
  # To avoid taking an outlier up, which is third highest FEV1 measurement
  rmax.append(O2_FEV1['FEV1'][mask].nlargest(3).iloc[-1])
# Add rmax to df_unblocked_factor with column name Unblocked FEV1 (L)
df_unblocked_factor['Unblocked FEV1 (L)'] = pd.Series(rmax)

# Compute Unblocked FEV1 as a percentage of lung damage
# Healthy FEV1 is the FEV1 Set As
df_unblocked_factor['Unblocked FEV1 as a % of lung damage'] = 100 * (1 - df_unblocked_factor['Unblocked FEV1 (L)'] / df_unblocked_factor['FEV1 Set As'])

# Plot a scatter of Unblocked FEV1 (&) against Age
def lung_damage_with_var(df, var, plotsdir):
  fig = px.scatter(df_unblocked_factor, x=var, y='Unblocked FEV1 as a % of lung damage')
  title="Lung damage with {} ({} individuals)".format(var, len(df_unblocked_factor))
  fig.update_layout(autosize=False, width=500, height=500, title=title)
  fig.update_traces(marker=dict(size=5),
                    selector=dict(mode='markers'))
  fig.show()
  fig.write_image("{}/Factors - {}.pdf".format(plotsdir, title), width=500, height=500)

lung_damage_with_var(df_unblocked_factor, 'Age', plotsdir)
lung_damage_with_var(df_unblocked_factor, 'Sex', plotsdir)

In [ ]:
# Factor function for FEV1
O2_FEV1_factors = merge_pred_ex_labels_to(O2_FEV1, get_pred_ex_labels(datadir))

df_unblocked_factor=pd.DataFrame(columns=['ID,', 'Unblocked FEV1 (L)', 'Healthy FEV1 (L)'])
for id in O2_FEV1.ID.unique():
  # For a given patient id, filter the FEV1 measurements
  mask = O2_FEV1['ID'] == id
  O2_FEV1_patient = O2_FEV1[mask]
  # Find the unblocked FEV1 (L). We assume that, over the 6 months study period, the patient has done some measurements where he was not blocked
  # To avoid taking an outlier up, which is third highest FEV1 measurement
  rmax=O2_FEV1_patient['FEV1'].nlargest(3).iloc[-1]
  # Get the theoretical healthy FEV1 (L)
  healthy_fev1=O2_FEV1_patient['FEV1 Set As'].iloc[0]
  # Add the patient id, reversed max FEV1 and healthy FEV1 (L) to the dataframe
  new_row = pd.DataFrame({'ID': [id], 'Unblocked FEV1 (L)': [rmax], 'Healthy FEV1 (L)': [healthy_fev1]})
  df_unblocked_factor = pd.concat([df_unblocked_factor, new_row])

# Left join O2_FEV1_factors with df_unblocked_factor on ID
O2_FEV1_factors = pd.merge(O2_FEV1_factors, df_unblocked_factor, on='ID', how='left')

# Plot unblocked fev1 (L) in x, measured fev1 in y, and color by Is Exacerbated
fig = px.scatter(O2_FEV1_factors, x='Unblocked FEV1 (L)', y='FEV1', color='Is Exacerbated', color_discrete_sequence=[get_stable_color(), get_ex_color()])
title="Impact of % small airways blockage on unblocked FEV1"
fig.update_layout(autosize=False, width=500, height=500, title=title)
fig.update_traces(marker=dict(size=5),
                  selector=dict(mode='markers'))
fig.show() 
# Save figure
fig.write_image("{}/Factors - {}.pdf".format(plotsdir, title), width=500, height=500)

### Exploration
#### Old factor function code

In [ ]:
# Factor function for unblocked FEV1 (L)
df_unblocked_factor=pd.DataFrame(columns=['ID,', 'Unblocked FEV1 (L)', 'Healthy FEV1 (L)'])
for id in O2_FEV1.ID.unique():
  # For a given patient id, filter the FEV1 measurements
  mask = O2_FEV1['ID'] == id
  O2_FEV1_patient = O2_FEV1[mask]
  # Find the unblocked FEV1 (L). We assume that, over the 6 months study period, the patient has done some measurements where he was not blocked
  # To avoid taking an outlier up, which is third highest FEV1 measurement
  rmax=O2_FEV1_patient['FEV1'].nlargest(3).iloc[-1]
  # Get the theoretical healthy FEV1 (L)
  healthy_fev1=O2_FEV1_patient['FEV1 Set As'].iloc[0]
  # Add the patient id, reversed max FEV1 and healthy FEV1 (L) to the dataframe
  new_row = pd.DataFrame({'ID': [id], 'Unblocked FEV1 (L)': [rmax], 'Healthy FEV1 (L)': [healthy_fev1]})
  df_unblocked_factor = pd.concat([df_unblocked_factor, new_row])


def add_lung_damage(fig, df, lung_damage_prct):
  xmax=df['Healthy FEV1 (L)'].max(); xmin=df['Healthy FEV1 (L)'].min()
  a=1-lung_damage_prct/100
  fig.add_shape(type="line", x0=xmin, y0=a*xmin, x1=xmax, y1=a*xmax, line=dict(color="Red", width=0.5))
  # Add line legend for no lung damage
  fig.add_annotation(x=1.02*xmax, y=a*xmax, text="{}%".format(lung_damage_prct), showarrow=False, font=dict(size=10, color="Red"))

# Plot a scatter of unblocked FEV1 (L) against healthy FEV1 (L)
fig = px.scatter(df_unblocked_factor, x='Healthy FEV1 (L)', y='Unblocked FEV1 (L)')
add_lung_damage(fig, df_unblocked_factor, 0)
add_lung_damage(fig, df_unblocked_factor, 50)
title="Impact of lung damage on healthy FEV1"
fig.update_layout(autosize=False, width=500, height=500, title="Impact of lung damage on healthy FEV1")
fig.update_traces(marker=dict(size=5),
                  selector=dict(mode='markers'))
fig.show()
# Save figure
fig.write_image("{}/Factors - {}.pdf".format(plotsdir, title), width=500, height=500)

In [ ]:
O2_FEV1_factors = merge_pred_ex_labels_to(O2_FEV1, get_pred_ex_labels(datadir))

df_unblocked_factor=pd.DataFrame(columns=['ID,', 'Unblocked FEV1 (L)', 'Healthy FEV1 (L)'])
for id in O2_FEV1.ID.unique():
  # For a given patient id, filter the FEV1 measurements
  mask = O2_FEV1['ID'] == id
  O2_FEV1_patient = O2_FEV1[mask]
  # Find the unblocked FEV1 (L). We assume that, over the 6 months study period, the patient has done some measurements where he was not blocked
  # To avoid taking an outlier up, which is third highest FEV1 measurement
  rmax=O2_FEV1_patient['FEV1'].nlargest(3).iloc[-1]
  # Get the theoretical healthy FEV1 (L)
  healthy_fev1=O2_FEV1_patient['FEV1 Set As'].iloc[0]
  # Add the patient id, reversed max FEV1 and healthy FEV1 (L) to the dataframe
  new_row = pd.DataFrame({'ID': [id], 'Unblocked FEV1 (L)': [rmax], 'Healthy FEV1 (L)': [healthy_fev1]})
  df_unblocked_factor = pd.concat([df_unblocked_factor, new_row])

# Left join O2_FEV1_factors with df_unblocked_factor on ID
O2_FEV1_factors = pd.merge(O2_FEV1_factors, df_unblocked_factor, on='ID', how='left')

# Plot unblocked fev1 (L) in x, measured fev1 in y, and color by Is Exacerbated
fig = px.scatter(O2_FEV1_factors, x='Unblocked FEV1 (L)', y='FEV1', color='Is Exacerbated', color_discrete_sequence=[get_stable_color(), get_ex_color()])
title="Impact of % small airways blockage on unblocked FEV1"
fig.update_layout(autosize=False, width=500, height=500, title=title)
fig.update_traces(marker=dict(size=5),
                  selector=dict(mode='markers'))
fig.show() 
# Save figure
fig.write_image("{}/Factors - {}.pdf".format(plotsdir, title), width=500, height=500)